In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install -r requirements.txt

## Build dataset

In [ ]:
from src.surfnet_train.data.data_processing import get_annotations_from_files, get_annotations_from_db
from src.surfnet_train.data.data_processing import generate_yolo_files, get_train_valid, build_yolo_annotations_for_images
from pathlib import Path

In [ ]:
from argparse import Namespace
args = Namespace(
    data_dir="/home/charles/Programs/surfrider/yolov5_train_test/data/test_data/",
    images_dir="/home/charles/Programs/surfrider/surfnet/data/images/images/",
    bbox_filename="bounding_boxes_202206171635.csv",
    images_filename="images_for_labelling_202206171635.csv",
    limit_data=40
)

In [ ]:
df_bboxes, df_images = get_annotations_from_files(Path(args.data_dir),
                                                  args.bbox_filename,
                                                  args.images_filename)

In [66]:
# images that were already processed
from src.surfnet_train.data.data_processing import find_img_ids_to_exclude
to_exclude = find_img_ids_to_exclude(args.data_dir)

In [69]:
yolo_filelist, cpos, cneg = build_yolo_annotations_for_images(args.data_dir, args.images_dir, 
                                                              df_bboxes, df_images, args.limit_data, to_exclude)

number of images in images folder: 5233
number of images referenced in database: 7484
number of images with a bbox in database: 6161
after exclusion, number of images with a bbox in database: 6121


In [ ]:
# split train test
train_files, val_files = get_train_valid(yolo_filelist)

In [ ]:
# generates data.yaml, train.txt, val.txt
generate_yolo_files(args.data_dir, train_files, val_files)

## Training

command to run in yolo folder

In [ ]:
python train.py --img 640 --hyp "data/hyps/hyp_surfnet.yaml" --batch 8 --epochs 2 --data "/datadrive/data/data_20062022/data.yaml" --weights "yolov5s.pt" --workers 7 --project "cha" --name "yolo_cha_test" --exist-ok